In [1]:
import envqa_dataset
import torch

### Load Dataset
#### Put env_qa_objects.h5 and env_qa_frame_obj_cls.h5 under data/ folder
#### See get_video_feature() function in VQAFeatureDataset for get video features

In [2]:
batch_size = 30

In [3]:
questions = json.load(open("data/train_full_question.json"))

In [5]:
train_dataset = envqa_dataset.VQAFeatureDataset('train', questions, anno_root="data/", 
                                                  img_root="data/", 
                                                  video_annotation="env_qa_video_annotations_v1.json",
                                                  predict_segment="env_qa_full_predicted_segment.json",
                                                  img_file='env_qa_objects.h5', max_frame=400, max_obj=30,
                                                  max_q_len=24, max_a_len=11, batch_size=30, if_gt_vision=False)

loading existing dictionaries from data/dictionaries.pkl
loading image features from env_qa_objects.h5 h5 file
data/env_qa_frame_obj_cls.h5
encode object name to word


100%|████████████████████████████████████████████████████████████████████████| 970151/970151 [02:24<00:00, 6719.42it/s]


loading completed
58086 questions before filtering
57437 questions after filtering
load dataset


100%|██████████████████████████████████████████████████████████████████████████| 57437/57437 [00:19<00:00, 2922.05it/s]


### Get Batch

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False,
                                           num_workers=0)

In [6]:
data_iter = iter(train_loader)
batch = next(data_iter)

### Decode object name

In [8]:
import h5py

In [9]:
# load object name file
object_labels = h5py.File("data/env_qa_frame_obj_cls.h5", "r")

# get an example
object_ids = [int(ix) for ix in object_labels['labels'][0][0]]
print(object_ids)

# use the provided dictionary to decode the object name
train_dataset.dict['obj_word_dict'].decodeSeq(object_ids)

[120, 71, 0]


['counter', 'top', '<UNK>']

### Transform an answer to role value format
the dataloader has already achived this function

In [10]:
print(questions[0]['answer'])
train_dataset.answer_to_role_value(questions[0]['answer'])

putting paper towel roll near counter top


{'action': 'putting',
 'subjects': 'paper towel roll',
 'prep': 'near',
 'objects': 'counter top',
 'state': 'NONE',
 'yes/no': 'NONE',
 'order': 'NONE',
 'number': 'NONE'}

### Evaluation

In [1]:
import json
from eval_env_qa import EnvQAEval

In [9]:
prediction = [{'question_id': 'FloorPlan325_physics_34_1Step_02_02',
              'answer': 'moving vase to side table'},
              {'question_id': 'FloorPlan206_physics_37_04',
               'answer': 'throwing vase'}]

# load test_full_question.json or test_full_question.json for evaluating the performance on test or val set
gt = [{'answer': 'moving vase to side table',
       'video_id': 'FloorPlan325_physics_34_1Step_02',
       'question': 'what happend in the video',
       'question_type': 'QueryEvent',
       'question_id': 'FloorPlan325_physics_34_1Step_02_02'},
      {'answer': 'breaking vase',
       'video_id': 'FloorPlan206_physics_37',
       'question': 'what happened, after throwing remote control to break television and before throwing bowl to break window',
       'question_id': 'FloorPlan206_physics_37_04',
       'question_type': 'QueryEvent'}
      ]

In [10]:
tsea_evalutor = EnvQAEval(gt, prediction)

In [11]:
tsea_evalutor.evaluate()

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1008.25it/s]


0.7500
